In [4]:
import numpy as np
import pandas as pd
import math
from sklearn import datasets
import pydotplus
from IPython.display import Image
from sklearn.tree import export_graphviz

# Load iris dataset
iris = datasets.load_iris()
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

# Function to find label for a value
def label(val, *boundaries):
    if val < boundaries[0]:
        return 'a'
    elif val < boundaries[1]:
        return 'b'
    elif val < boundaries[2]:
        return 'c'
    else:
        return 'd'

# Function to convert a continuous data into labelled data
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second) / 2
    third = (df[old_feature_name].max() + second) / 2
    return df[old_feature_name].apply(label, args=(first, second, third))

# Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df.drop(['sl', 'sw', 'pl', 'pw'], axis=1, inplace=True)

class TreeNode:
    def __init__(self, data, output):
        self.data = data
        self.children = {}
        self.output = output
        self.index = -1

    def add_child(self, feature_value, obj):
        self.children[feature_value] = obj

class BuildTree:
    def __init__(self):
        self.__root = None

    def __frequency_counter(self, Y):
        d = {}
        for i in Y:
            if i in d:
                d[i] += 1
            else:
                d[i] = 1
        return d

    def __entropy(self, Y):
        freq_map = self.__frequency_counter(Y)
        entropy_ = 0
        total = len(Y)
        for i in freq_map:
            p = freq_map[i] / total
            entropy_ += (-p) * math.log2(p)
        return entropy_

    def __gain_ratio(self, X, Y, selected_feature):
        original = self.__entropy(Y)
        entropy_after_splitting = 0
        split_info = 0
        values = set(X[:, selected_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        starting_size = df.shape[0]
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            entropy_after_splitting += (current_size / starting_size) * self.__entropy(df1[df1.shape[1] - 1])
            split_info += (-current_size / starting_size) * math.log2(current_size / starting_size)

        if split_info == 0:
            return math.inf

        info_gain = original - entropy_after_splitting
        gain_ratio = info_gain / split_info
        return gain_ratio

    def __decision_tree(self, X, Y, features, level, classes, all_features=np.array([i for i in df.columns])):
        if len(features) == 0:
            print(f"Level {level}")
            freqs = self.__frequency_counter(Y)
            output = None
            max_count = -99999999999999
            for i in classes:
                count = freqs.get(i, 0)
                if count > max_count:
                    output = i
                    max_count = count
                print(f"Count of {i} = {count}")
            print(f"Current Entropy is = {self.__entropy(Y)}")
            print("Reached leaf Node")
            print()
            return TreeNode(None, output)

        if len(set(Y)) == 1:
            print(f"Level {level}")
            output = list(set(Y))[0]
            for class_ in classes:
                count = sum(Y == class_)
                print(f"Count of {class_} = {count}")
            print("Current Entropy is =  0.0")
            print("Reached leaf Node")
            print()
            return TreeNode(None, output)

        max_gain = -math.inf
        final_feature = None
        for f in features:
            current_gain = self.__gain_ratio(X, Y, f)
            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print(f"Level {level}")
        freqs = self.__frequency_counter(Y)
        output = None
        max_count = -math.inf
        for class_ in classes:
            count = freqs.get(class_, 0)
            if count > max_count:
                output = class_
                max_count = count
            print(f"Count of {class_} = {count}")
        print(f"Current Entropy is = {self.__entropy(Y)}")
        print(f"Splitting on feature {all_features[final_feature]} with gain ratio {max_gain}")
        print()

        unique_values = set(X[:, final_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        current_node = TreeNode(final_feature, output)
        index = features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
            df_new = df[df[final_feature] == i]
            node = self.__decision_tree(df_new.iloc[:, 0:df_new.shape[1] - 1].values,
                                        df_new.iloc[:, df_new.shape[1] - 1].values, features, level + 1, classes)
            current_node.add_child(i, node)
        features.insert(index, final_feature)
        return current_node

    def fit(self, X, Y):
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        self.__root = self.__decision_tree(X, Y, features, level, classes)

    def __predict_for_single_point(self, data, node):
        if len(node.children) == 0:
            return node.output

        val = data[node.data]
        if val not in node.children:
            return node.output

        return self.__predict_for_single_point(data, node.children[val])

    def predict(self, X):
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for_single_point(X[i], self.__root)
        return Y

    def score(self, X, Y):
        Y_predicted = self.predict(X)
        counter = 0
        for i in range(len(Y_predicted)):
            if Y_predicted[i] == Y[i]:
                counter += 1
        return counter / len(Y_predicted)

    def export_to_graphviz(self, feature_names):
        def add_node(graph, node, parent_name=None, edge_label=None):
            if node.data is not None:
                feature_name = feature_names[node.data]
                graph_node = f"{feature_name}\nEntropy={node.output}"
            else:
                graph_node = f"Leaf\nClass={node.output}"

            graph.add_node(pydotplus.graphviz.Node(graph_node))

            if parent_name is not None:
                graph.add_edge(pydotplus.graphviz.Edge(parent_name, graph_node, label=edge_label))

            for feature_value, child in node.children.items():
                add_node(graph, child, graph_node, str(feature_value))

        graph = pydotplus.graphviz.Dot(graph_type='digraph')
        add_node(graph, self.__root)
        return graph

from sklearn.model_selection import train_test_split

x = df.values
y = iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y)
clf = BuildTree()
clf.fit(x_train, y_train)

graph = clf.export_to_graphviz(df.columns)
graph.write_png("tree.png")
Image(graph.create_png())

print()
print("Score =", clf.score(x_test, y_test))


Level 0
Count of 0 = 34
Count of 1 = 38
Count of 2 = 40
Current Entropy is = 1.581711119299905
Splitting on feature pw_labeled with gain ratio 0.6867489104652664

Level 1
Count of 0 = 0
Count of 1 = 28
Count of 2 = 11
Current Entropy is = 0.858230792641141
Splitting on feature pl_labeled with gain ratio 0.3884273222232152

Level 2
Count of 0 = 0
Count of 1 = 27
Count of 2 = 6
Current Entropy is = 0.6840384356390417
Splitting on feature sl_labeled with gain ratio 0.13965691626803747

Level 3
Count of 0 = 0
Count of 1 = 17
Count of 2 = 5
Current Entropy is = 0.7732266742876346
Splitting on feature sw_labeled with gain ratio 0.12498985492247726

Level 4
Count of 0 = 0
Count of 1 = 5
Count of 2 = 0
Current Entropy is = 0.0
Reached leaf Node

Level 4
Count of 0 = 0
Count of 1 = 2
Count of 2 = 0
Current Entropy is = 0.0
Reached leaf Node

Level 4
Count of 0 = 0
Count of 1 = 10
Count of 2 = 5
Current Entropy is = 0.9182958340544896
Reached leaf Node

Level 3
Count of 0 = 0
Count of 1 = 0
Coun

InvocationException: GraphViz's executables not found

In [3]:
pip install GraphViz


Note: you may need to restart the kernel to use updated packages.
